In [433]:
import numpy as np
import copy
import time

In [434]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [435]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [436]:
std0 = torch.eye(3, dtype=torch.float64) * 0.3
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(3, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", torch.zeros(3, dtype=torch.float64))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

In [437]:
X, Y, beta = generate_stdp_dataset(3, 10, 0, 1)

In [438]:
print(X)
print(Y)
print(beta)

[[0.1091475  0.02707316 0.22884823]
 [0.0131476  0.38097679 0.31598663]
 [0.55345836 0.50659274 0.18688269]
 [0.18478382 0.40802562 0.17913404]
 [0.40246588 0.0294645  0.13748655]
 [0.25378962 0.34640381 0.26353375]
 [0.20958597 0.47822705 0.43528963]
 [0.38924836 0.06346602 0.34210274]
 [0.27431815 0.00857661 0.26379397]
 [0.10908667 0.21053985 0.44512697]
 [0.3721208  0.18168442 0.34376967]]
[-0.57500573  0.12161552  0.13190995  0.45732652 -0.29505838 -0.17226027
 -0.33994132  0.35393253  0.23767532 -0.03356426  0.11337013]
[0.71402107 0.03418198 0.11366686]


In [439]:
X_train = copy.deepcopy(X)
Y_train = copy.deepcopy(Y[:len(Y) - 1])
X_train = [torch.tensor(member) for member in X_train]
Y_train = [torch.tensor(member) for member in Y_train]

In [440]:
np.std(Y_train)

0.3139843843608481

In [441]:
def train_SVI(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        svi.step(D_hat)
    
    breakpoint()
    mu_q = pyro.param("mu_q")
    return mu_q

In [442]:
z = [0 for i in range(len(Y_train) + 1)]
print(z)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [443]:
y_hat = max(Y_train)
y_bottom = min(Y_train)
print(y_hat)
print(y_bottom)
conformal_set = []
start = time.time()
while y_hat >= y_bottom:
#     print(f"---- New Loop (y_hat = {y_hat}) ----")
    pyro.clear_param_store()
    D_hat = [(X_train[i], Y_train[i]) for i in range(len(Y_train))]
    D_hat.append((X_train[-1], y_hat))
    mu_q = train_SVI(D_hat, 10)
    # mu_q * x_i - y_i
    rank = [sum(D_hat[i][0] * mu_q) - D_hat[i][1] for i in range(len(D_hat))]
    
    y_hat_rank = rank[-1]
    count = 0
    for member in rank:
        if member > y_hat_rank:
            count += 1
    if count / len(D_hat_y) > 0.1:
        # y_hat is conformal
        conformal_set.append(copy.deepcopy(y_hat))
    else:
        print(f"{y_hat} Not added")
    y_hat -= 0.01
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

tensor(0.4573, dtype=torch.float64)
tensor(-0.5750, dtype=torch.float64)
-0.3826734750350787 Not added
-0.4026734750350787 Not added
-0.42267347503507874 Not added
-0.43267347503507875 Not added
-0.45267347503507877 Not added
-0.4626734750350788 Not added
-0.5126734750350788 Not added
-0.5226734750350788 Not added
-0.5526734750350788 Not added


In [445]:
print(conformal_set)
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")

[tensor(-0.5727, dtype=torch.float64), tensor(0.4573, dtype=torch.float64)]
Y[-1] is covered
Elapsed Time: 9.105756998062134
